In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.resnet import preprocess_input

In [ ]:
proj_path = 'C:/Users/babes/ABE4662/FinalProject/'
train_dir = os.path.join(proj_path, 'TrainingImages')
validation_dir = os.path.join(proj_path, 'ValidationImages')

train_whiteOak_dir = os.path.join(train_dir, 'Quercus alba')
train_loblollyPine_dir = os.path.join(train_dir, 'Pinus taeda')
validation_whiteOak_dir = os.path.join(validation_dir, 'Quercus alba')
validation_loblollyPine_dir = os.path.join(validation_dir, 'Pinus taeda')

num_whiteOak_tr = len(os.listdir(train_whiteOak_dir))
num_loblollyPine_tr = len(os.listdir(train_loblollyPine_dir))

num_whiteOak_val = len(os.listdir(validation_whiteOak_dir))
num_loblollyPine_val = len(os.listdir(validation_loblollyPine_dir))

total_train = num_whiteOak_tr + num_loblollyPine_tr
total_val = num_whiteOak_val + num_loblollyPine_val

In [ ]:
image_size = 448
batch_size = 12 # 24
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                             horizontal_flip=True,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2)

#train_path='C:/Users/babes/ABE4662/FinalProject/TrainingImages'
train_generator = data_generator_with_aug.flow_from_directory(train_dir, 
                                                     target_size=(image_size, image_size), 
                                                     batch_size=batch_size,
                                                     class_mode='categorical')

data_generator_with_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_path='C:/Users/babes/ABE4662/FinalProject/ValidationImages'
validation_generator = data_generator_with_no_aug.flow_from_directory(validation_dir,
                                                          target_size=(image_size, image_size),
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

In [ ]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

num_classes = 2
model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
model.layers[0].trainable = False

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 20
history = model.fit_generator(
    train_generator,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_val // batch_size
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

In [ ]:
#model.fit_generator(train_generator, 
#steps_per_epoch=3, 
#epochs=20, 
#validation_data=validation_generator, 
#validation_steps=1)

In [ ]:
import numpy as np
from os.path import join
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

image_dir = proj_path+'TestImages'
img_paths = [join(image_dir, filename) for filename in 
               ['Pinus taeda/pine1.jpg',
                'Pinus taeda/pine2.jpg',
                'Pinus taeda/pine3.jpg',
                'Pinus taeda/pine4.jpg',
                'Pinus taeda/pine5.jpg',
                'Quercus alba/oak1.jpg',
                'Quercus alba/oak2.jpg',
                'Quercus alba/oak3.jpg',
                'Quercus alba/oak4.jpg',
                'Quercus alba/oak5.jpg',
                'Quercus alba/oak6.jpg',
                'Quercus alba/oak7.jpg']]

image_size = 224
def read_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)

    return output


test_data = read_prep_images(img_paths)


In [ ]:
preds = model.predict(test_data)
print(preds)

In [ ]:
import json
from IPython.display import Image, display

def decode_predictions(preds, top=5, class_list_path=None):
  if len(preds.shape) != 2 or preds.shape[1] != 2:
    raise ValueError('`decode_predictions` expects '
                     'a batch of predictions '
                     '(i.e. a 2D array of shape (samples, 1000)). '
                     'Found array with shape: ' + str(preds.shape))
  index_list = json.load(open(class_list_path))
  results = []
  for pred in preds:
    top_indices = pred.argsort()[-top:][::-1]
    result = [tuple(index_list[str(i)]) + (pred[i],) for i in top_indices]
    result.sort(key=lambda x: x[2], reverse=True)
    results.append(result)
  return results


most_likely_labels = decode_predictions(preds, top=5, class_list_path='C:/Users/babes/ABE4662/FinalProject/class_tree_species.json')

for i, img_path in enumerate(img_paths):
    display(Image(img_path))
    print(most_likely_labels[i])
